In [1]:
#generate data do compare network quality
import sys
sys.path.insert(0, '../../helperScripts/')
from helperFunctions import *
import scmra
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import pandas as pd
from datetime import datetime
import pickle
import seaborn as sns; sns.set_theme()
from matplotlib.pyplot import figure
import scmra

def print_rloc(rlocDict, key, fileLocation):
  rloc = rlocDict.rloc[key]
  rloc = reorder_rloc(rloc)
  rloc.to_csv(fileLocation, sep = "\t")

In [20]:
###TESTING SOME OF THE SIMULATIONS FOR MRA? CNR COMPARISON

#RAS SIMULATIONS

def run_cnr(cellPopulationList, noise, cell, count, result, rglob, rtot, cell_annot, tx_annot, group_annot, theta):
    
    #theta = THETA
    #scd = scmra.ScData(rglob=rglob, rtot = rtot, cell_annot=cell_annot, tx_annot=tx_annot, group_annot= group_annot)

    eta, scCnrResult, edgeNumber = get_suitable_solution_fast(rglob, rtot, cell_annot, tx_annot, group_annot, 
                etaGuesses=30, reconstructionType = "CNR", theta = theta)

    #get number of different edges
    vars_lst = [var for var in scCnrResult.vardict if var.startswith('IDev')]
    vars_vals = [scCnrResult.vardict[v] for v in vars_lst]
    lengthIdcs = (np.count_nonzero(vars_vals))
    numDiffEdges = lengthIdcs

    #make MRA simulation
    #p = scmra.ScCnrProblem(scd, eta=eta, theta=theta) 
    #p.cpx.solve()
    #s = scmra.ScCnrResult(p)
    s = scCnrResult

    populationString = ""
    for population in cellPopulationList:
        populationString += population
        populationString += ','
    populationString = populationString[:-1]

    residuals = 0
    n_res = np.size(scCnrResult.residuals_complete) + \
            np.size(scCnrResult.residuals_incomplete)
    residuals += np.sum(np.array(np.square(scCnrResult.residuals_complete)))/n_res
    residuals += np.sum(np.array(np.square(scCnrResult.residuals_incomplete)))/n_res

    result.add_parameter([noise, cell, eta, count, edgeNumber, "CNR", populationString, numDiffEdges, theta, residuals])

    #add simulation output to result
    result.add_result(s)

def run(cellPopulation, noise, cell, count, result, rglob, rtot, cell_annot, tx_annot, PopulationID):
    edgesToReconstruct = 13
    if(cellPopulation == "ras"):
        edgesToReconstruct = 12
    eta, scMraResult, edgeNumber = get_suitable_solution_fast(rglob, rtot, cell_annot, tx_annot, etaGuesses=30, expectedEdges = edgesToReconstruct)
    
    #add simulation output to result
    result.add_result(scMraResult)

    #add values to result
    numDiffEdges = 0
    residuals = 0
    n_res = np.size(scMraResult.residuals_complete) + \
            np.size(scMraResult.residuals_incomplete)
    residuals += np.sum(np.array(np.square(scMraResult.residuals_complete)))/n_res
    residuals += np.sum(np.array(np.square(scMraResult.residuals_incomplete)))/n_res
    result.add_parameter([noise, cell, eta, count, edgeNumber, "MRA", PopulationID, numDiffEdges, "NO_THETA",residuals])

header = ['NOISE', 'CELLNUM', 'ETA', 'REPEAT', 'EDGES', 'MODELTYPE','POPULATIONS','NUMDIFFEDGES','THETA','RESIDUALS']
result = MRASimulationClass(header)
NOISE = 0.2
CELLNUM = 500
rglobWT, rtotWT, rglobMUT, rtotMUT = generate_raw_data_for_MRA_and_CNR(CELLNUM, NOISE, "braf")
#for this simulation we add no treatment/ perturbations
cell_annot = None
tx_annot = None
count = 0 #for repeats
#combine the mutant & wt data for CNR
rglobList = [rglobWT, rglobMUT]
rtotList = [rtotWT, rtotMUT]
#make group annot: 'wt' ->'cellIds', , 'braf' ->'cellIds'
group_annot = {}
group_annot["wt"] = list(rglobWT.columns)
group_annot["braf"] = list(rglobMUT.columns)
#combine rglob and rtot
rglob = pd.concat(rglobList, axis=1)
rtot = pd.concat(rtotList, axis=1) 
           
run_cnr(["wt","braf"],NOISE, CELLNUM, count, result, rglob, rtot, cell_annot, tx_annot, group_annot, 0.02)

run("wt",NOISE, CELLNUM, count, result, rglobWT, rtotWT, cell_annot, tx_annot, "WT:wt_braf")
run("braf",NOISE, CELLNUM, count, result, rglobMUT, rtotMUT, cell_annot, tx_annot, "BRAF:wt_braf")

estimated alpha is: 1.0
DOES NOT EXIST

------

None
------

None
------

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 73.137912 after 0.02 sec. (2.05 ticks)
Tried aggregator 2 times.
MIQP Presolve eliminated 0 rows and 1 columns.
MIQP Presolve modified 121 coefficients.
Aggregator did 462 substitutions.
Reduced MIQP has 11704 rows, 12639 columns, and 133991 nonzeros.
Reduced MIQP has 1155 binaries, 0 generals, 0 SOSs, and 1386 indicators.
Reduced MIQP objective Q matrix has 11000 nonzeros.
Presolve time = 0.14 sec. (61.83 ticks)
Probing time = 0.17 sec. (1.50 ticks)
Tried aggregator 2 times.
Aggregator did 121 substitutions.
Reduced MIQP has 11583 rows, 12518 columns, and 133749 nonzeros.
Reduced MIQP has 1155 binaries, 0 generals, 0 SOSs, and 1386 indicators.
Reduced MIQP objective Q matrix has 11000 nonzeros.
Presolve time = 0.17 sec. (58.23 ticks)
Classifier predicts products in MIQP should be line

In [23]:

print("RMSE RLOC WT MRA:")
print(calc_rmse(rloc_true, result.result[1].rloc))

print("RMSE RLOC BRAF MRA:")
print(calc_rmse(rloc_braf, result.result[2].rloc))

print("RMSE RLOC WT CNR:",calc_rmse(rloc_true, result.result[0].rloc["wt"]))
print("RMSE RLOC BRAF CNR:",calc_rmse(rloc_braf, result.result[0].rloc["braf"]))

print(rloc_true)
print("_______MRA_________")
print(result.result[1].rloc)
print(result.result[2].rloc)
print("_______CNR_________")
print(result.result[0].rloc["wt"])
print(result.result[0].rloc["braf"])


print("_______________________________")


#Test DIFF RMSE calculation for CNR/ MRA
def calculate_rmse_of_network_differences(rlocList, populationStringList, diffDict):

   rloc_wt = None
   rloc_mut = None

   for popIdx in range(len(populationStringList)):
      pop = populationStringList[popIdx]
      if(pop == "wt"):
         rloc_wt = rlocList[popIdx]
      else:
         mutString = pop
         rloc_mut = rlocList[popIdx]

   assert(rloc_wt is not None)
   assert(rloc_mut is not None)

   #true difference rloc matrix
   trueDiff = diffDict[mutString]

   #reconstructed difference rloc matrix
   rloc_wt = reorder_rloc(rloc_wt)
   rloc_mut = reorder_rloc(rloc_mut)
   reconstructedDiff = ((rloc_wt - rloc_mut)**2)**0.5

   #from these final two rlocs calculate RMSE
   trueDiff = reorder_rloc(trueDiff)
   reconstructedDiff = reorder_rloc(reconstructedDiff)

   print(trueDiff)
   print(reconstructedDiff)
   rmse = calc_rmse(trueDiff, reconstructedDiff)
   return(rmse)

dictX = {}
dictX['braf'] = ((rloc_true - rloc_braf)**2)**0.5
dictX['ras'] = ((rloc_true - rloc_ras)**2)**0.5

a = calculate_rmse_of_network_differences([result.result[0].rloc['wt'], result.result[0].rloc["braf"]], ["braf", "wt"], dictX)
b = calculate_rmse_of_network_differences([result.result[1].rloc, result.result[2].rloc], ["braf", "wt"], dictX)

print(a)
print(b)


RMSE RLOC WT MRA:
0.07445806371048029
RMSE RLOC BRAF MRA:
0.09084319667482528
RMSE RLOC WT CNR: 0.08046867883808491
RMSE RLOC BRAF CNR: 0.08863577924879751
             Sos       Ras      Raf1       Mek      Erk    P90Rsk      PI3K  \
Sos    -1.000000  0.000000  0.000000  0.000000  0.00000 -0.807052  0.000000   
Ras     0.999877 -1.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
Raf1    0.000000  1.001393 -1.000000  0.000000  0.00000  0.000000  0.000000   
Mek     0.000000  0.000000  0.087174 -1.000000  0.00000  0.000000  0.000000   
Erk     0.000000  0.000000  0.000000  0.991678 -1.00000  0.000000  0.000000   
P90Rsk  0.000000  0.000000  0.000000  0.000000  0.95669 -1.000000  0.000000   
PI3K    0.000000  0.000176  0.000000  0.000000  0.00000  0.000000 -1.000000   
Akt     0.000000  0.000000  0.000000  0.000000  0.00000  0.000000  0.396081   
C3G     0.000000  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
Rap1    0.000000  0.000000  0.000000  0.000000  0.0000

In [7]:

#GET RLOCS FOR THE MRA CNR COMPARISON

pickleFile = '/DATA/t.stohn/MRA/scMRA-analysis/notebooks/paperAnalyses/bin/SIMULATION_CNR_MRA_COMPARISON_10-06-2022|21:44:53.pickle'
result = pickle.load(open(pickleFile, 'rb'))

#get CNR diff rlocs
valueList = ["CNR", 0, 500, ",ras,wt"]
keyList = ["MODELTYPE", "NOISE", "CELLNUM", "POPULATIONS"]
resultObject = result.getResult(keyList, valueList, False) #the result from the simulation

#get a CNR network with RMSE_rloc_wt > 0.06 and one <0.0025
for obj in resultObject:
  rmse = calc_rmse(rloc_true, obj.rloc["wt"])
  if(rmse > 0.06):
    print_rloc(obj, "wt","/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_cnr_wt_badReconstruction.tsv" )
    print_rloc(obj,  "ras","/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_cnr_ras_badReconstruction.5.tsv" )
for obj in resultObject:
  rmse = calc_rmse(rloc_true, obj.rloc["wt"])
  if(rmse < 0.0025):
    print_rloc(obj,  "wt","/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_cnr_wt_goodReconstruction.tsv" )
    print_rloc(obj,  "ras","/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_cnr_ras_goodReconstruction.tsv" )


In [7]:
#GET RLOCS FOR THE PERFORMANCE ANALYSIS OF CNR
pickleFile = "/DATA/t.stohn/MRA/scMRA-analysis/notebooks/paperAnalyses/bin/SIMULATION_CNR_Noise_Cellnum_3_10-12-2022|07:46:14.pickle"
result = pickle.load(open(pickleFile, 'rb'))

#get CNR result objects with its rlocs
valueList = ["CNR", 0.5, 100, ",braf,wt"]
keyList = ["MODELTYPE", "NOISE", "CELLNUM", "POPULATIONS"]
resultObject = result.getResult(keyList, valueList, False) #the result from the simulation

worst_rmse = 0
worst_rloc_obj = None
for obj in resultObject:
  rmse = calc_rmse(rloc_true, obj.rloc["wt"])
  if(rmse > worst_rmse):
    worst_rloc_obj = obj

print_rloc(obj,  "wt","/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_cnr_wt_noise0.5_cellnum100.tsv" )
print_rloc(obj,  "braf","/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_cnr_braf_noise0.5_cellnum100.tsv" )